In [6]:
# Alright lets see what this is all about
import numpy as np
import pandas as pd
import sklearn 



In [8]:
#LOAD GIS WATERMAIN AND BREAK DATA
df = pd.read_csv('Kirk_All.csv')
#df.dropna(inplace=True) # Are there any columns where we can deal with NA values?
# Can we use imputation instead?
# breaks = pd.read_csv('Kirk_Break_Data.csv')
df['INSTYEAR'] = df['INSTYEAR'].map(lambda x: int((str(x)[-12:-8])), na_action='ignore')
#df = pd.get_dummies(df)
df.head()

,FID,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,SLP_CLASS,FULL_STRNA,SPEED,FUNC_CLASS
0,0,1,FIRE,2.0,8.0,UNK,NaN,Ur,6000,2.0,120TH AVE NE,25,MINOR ARTERIAL
1,1,2,FIRE,2.0,2.0,UNK,1992.0,AgC,7000,1.0,122ND LN NE,25,NEIGHBORHOOD ACCESS
2,2,3,FIRE,1.0,0.0,UNK,NaN,AgC,6000,0.0,124TH AVE NE,35,PRINCIPAL ARTERIAL
3,3,4,FIRE,3.0,6.0,DI,2005.0,Ur,4000,0.0,LAKE WASHINGTON BLVD NE,35,PRINCIPAL ARTERIAL
4,4,5,FIRE,149.0,8.0,AC,1963.0,Ur,7000,6.0,,,


In [13]:
# EXTRACTING FEATURES
features = ['FID_WA_Mai', 'TYPE', 'LENGTH', 'DIAMETER', 'MATERIAL', 'INSTYEAR', 'MUSYM', 'Dist_Rail', 'FULL_STRNA' , 'SLP_CLASS', 'SPEED', 'FUNC_CLASS']
pipes = df[features]
pipes

,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SLP_CLASS,SPEED,FUNC_CLASS
0,1,FIRE,2.0,8.0,UNK,NaN,Ur,6000,120TH AVE NE,2.0,25,MINOR ARTERIAL
1,2,FIRE,2.0,2.0,UNK,1992.0,AgC,7000,122ND LN NE,1.0,25,NEIGHBORHOOD ACCESS
2,3,FIRE,1.0,0.0,UNK,NaN,AgC,6000,124TH AVE NE,0.0,35,PRINCIPAL ARTERIAL
3,4,FIRE,3.0,6.0,DI,2005.0,Ur,4000,LAKE WASHINGTON BLVD NE,0.0,35,PRINCIPAL ARTERIAL
4,5,FIRE,149.0,8.0,AC,1963.0,Ur,7000,,6.0,,
...,...,...,...,...,...,...,...,...,...,...,...,...
41816,3963,DIST,1.0,12.0,DI,1971.0,No,9000,120TH AVE NE,0.0,25,COLLECTOR
41817,5622,DIST,1.0,8.0,DI,2003.0,AgC,10000,117TH AVE NE,2.0,25,NEIGHBORHOOD ACCESS
41818,5623,DIST,1.0,8.0,DI,2003.0,AgC,10000,117TH AVE NE,2.0,25,NEIGHBORHOOD ACCESS
41819,1093,DIST,83.0,12.0,DI,2009.0,AgC,5000,NE 115TH PL,0.0,35,COLLECTOR


In [77]:
breaks

,Date,Location,Zone,Size,Type,Comment,Year,Install_Year
0,1/8,347 7th Ave,315,6,CI,NaN,2004,NaN
1,1/9,696 16th Ave W,315,8,AC,NaN,2004,NaN
2,1/21,740 16th Ave W,315,6,AC,NaN,2004,NaN
3,2/25,10132 NE 59th,285,8,AC,Leak,2004,NaN
4,3/16,18th Ave & 6th St.,450,12,AC,Leak,2004,NaN
...,...,...,...,...,...,...,...,...
63,10/31/17,10830 NE 68th St,450,8,AC,Full Break,2017,NaN
64,5/23/18,11828 NE 75th St,650,6,AC,Bead Break,2018,NaN
65,5/30/19,11828 NE 75th St,650,6,AC,Break at abandoned service tap,2019,NaN
66,6/3/19,10425 NE 58th St,450,6,AC,Full Break,2019,NaN


In [10]:
# GROUP PIPES BY THEIR UNIQUE ID - NEEDED BECAUSE OF HOW SLOPE BREAKS UP PIPES IN DIFFERENT SLOPE CLASSES
def group_by_id(df):

    grouping = df.groupby('FID_WA_Mai').groups
    unique_pipes = df['FID_WA_Mai'].unique()
    indices = []
    for i in range(len(unique_pipes)):
        pipe = unique_pipes[i]
        val = grouping[pipe][0]
        indices.append(val)
    
    # CALCULATE AVERAGE SLOPES FOR PIPES THAT GOT SPLIT INTO MULTIPLE ROWS
    slopes = df[['FID_WA_Mai', 'SLP_CLASS']]
    slopes = slopes.groupby('FID_WA_Mai')['SLP_CLASS'].mean()
    slopes = pd.Series(slopes)

    filtered = df.loc[indices, 'FID_WA_Mai':'FUNC_CLASS']
    filtered = filtered.drop('SLP_CLASS', axis=1)
    filtered['SLOPE'] = slopes.values

    return filtered

In [14]:
pipes = group_by_id(pipes)
pipes

,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
0,1,FIRE,2.0,8.0,UNK,NaN,Ur,6000,120TH AVE NE,25,MINOR ARTERIAL,2.000000
1,2,FIRE,2.0,2.0,UNK,1992.0,AgC,7000,122ND LN NE,25,NEIGHBORHOOD ACCESS,1.000000
2,3,FIRE,1.0,0.0,UNK,NaN,AgC,6000,124TH AVE NE,35,PRINCIPAL ARTERIAL,0.000000
3,4,FIRE,3.0,6.0,DI,2005.0,Ur,4000,LAKE WASHINGTON BLVD NE,35,PRINCIPAL ARTERIAL,0.000000
4,5,FIRE,149.0,8.0,AC,1963.0,Ur,7000,,,,3.888889
...,...,...,...,...,...,...,...,...,...,...,...,...
41801,7742,DIST,15.0,8.0,DI,2017.0,AgD,5000,96TH AVE NE,25,NEIGHBORHOOD ACCESS,0.785714
41804,7743,DIST,296.0,8.0,AC,1943.0,AgC,9000,NE 68TH ST,25,MINOR ARTERIAL,4.000000
41816,3963,DIST,1.0,12.0,DI,1971.0,No,9000,120TH AVE NE,25,COLLECTOR,4.000000
41817,5622,DIST,1.0,8.0,DI,2003.0,AgC,10000,117TH AVE NE,25,NEIGHBORHOOD ACCESS,3.000000


In [135]:
break_1_8_04 = filtered[(filtered['FULL_STRNA'] == '7TH AVE') & (filtered['INSTYEAR'].equals(2003))]
break_1_8_04

,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE


In [204]:
break_1_9_04 = filtered[(filtered['FULL_STRNA'] == '16TH AVE W')] #& (filtered['INSTYEAR'] < 2005) & (filtered['INSTYEAR'] > 2003)]
break_1_9_04 = break_1_9_04[break_1_9_04['INSTYEAR'] == 2004]
break_1_9_04

nearby = filtered[(filtered['FULL_STRNA'] == '16TH AVE W') & (filtered['INSTYEAR'] != 2004) & (filtered['Dist_Rail'] == 15000) & (filtered['DIAMETER'] == 8.0)] 
nearby

similar = filtered[(filtered['DIAMETER'] == 8.0) & (filtered['MATERIAL'] == 'AC') & (filtered['MUSYM'] == 'AmC')]
print(similar['FULL_STRNA'].unique())

similar = similar[(similar['FULL_STRNA'] == '17TH AVE W')]
similar

['10TH AVE' 'KIRKLAND CIR' '4TH ST' '17TH AVE W' '9TH AVE' '5TH AVE'
 '3RD ST' 'KIRKLAND WAY' '8TH LN' '15TH AVE' 'KIRKLAND AVE' 'NE 68TH ST'
 '19TH AVE' '8TH AVE' 'NE 45TH ST' ' ' '1ST ST' 'NE 44TH ST' 'NE 68TH PL'
 '6TH ST' '106TH AVE NE' '6TH AVE' '5TH ST' '7TH AVE' '109TH PL NE'
 '10TH AVE S' '5TH AVE S' 'LAKEVIEW DR' '5TH ST S' '6TH ST S' 'STATE ST S']


,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
3851,709,DIST,34.0,8.0,AC,1968,AmC,14000,17TH AVE W,25,NEIGHBORHOOD ACCESS,0.5


In [205]:
break_1_21_04 = filtered[(filtered['FULL_STRNA'] == '16TH AVE W') & (filtered['INSTYEAR'] == 2004)]
break_1_21_04

nearby = filtered[(filtered['FULL_STRNA'] == '16TH AVE W') & (filtered['INSTYEAR'] != 2004)] #& (filtered['Dist_Rail'] == 15000) & (filtered['DIAMETER'] == 6.0)] 


similar = filtered[(filtered['DIAMETER'] == 8.0) & (filtered['MATERIAL'] == 'AC') & (filtered['MUSYM'] == 'AmC')]
print(similar['FULL_STRNA'].unique())

similar = similar[(similar['FULL_STRNA'] == '17TH AVE W')]
similar

['10TH AVE' 'KIRKLAND CIR' '4TH ST' '17TH AVE W' '9TH AVE' '5TH AVE'
 '3RD ST' 'KIRKLAND WAY' '8TH LN' '15TH AVE' 'KIRKLAND AVE' 'NE 68TH ST'
 '19TH AVE' '8TH AVE' 'NE 45TH ST' ' ' '1ST ST' 'NE 44TH ST' 'NE 68TH PL'
 '6TH ST' '106TH AVE NE' '6TH AVE' '5TH ST' '7TH AVE' '109TH PL NE'
 '10TH AVE S' '5TH AVE S' 'LAKEVIEW DR' '5TH ST S' '6TH ST S' 'STATE ST S']


,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
3851,709,DIST,34.0,8.0,AC,1968,AmC,14000,17TH AVE W,25,NEIGHBORHOOD ACCESS,0.5


In [197]:
break_2_25_04 = filtered[(filtered['FULL_STRNA'] == 'NE 59TH ST')]
break_2_25_04

# THIS WILL LIKELY NEED TO BE altered in data set

,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
22011,3634,DIST,1.0,8.0,AC,1976,KpB,7000,NE 59TH ST,25,NEIGHBORHOOD ACCESS,1.000000
22012,3635,DIST,5.0,12.0,AC,1971,KpB,7000,NE 59TH ST,25,NEIGHBORHOOD ACCESS,1.000000
39310,6927,DIST,261.0,8.0,AC,1976,KpB,7000,NE 59TH ST,25,NEIGHBORHOOD ACCESS,2.586207


In [53]:
# Find pipes on NE 73RD ST installed in 2008
break_1_24_08 = pipes[(pipes['FULL_STRNA'] == 'NE 73RD ST') & (pipes['INSTYEAR'] == 2008)] 
# Find pipes installed before 2008 with similar attributes to find when broken pipes was installed
near1 = pipes[(pipes['FULL_STRNA'] == 'NE 73RD ST') & (pipes['INSTYEAR'] < 2008) & (pipes["Dist_Rail"] == 8000) & (pipes["DIAMETER"] == 6) & (pipes["MATERIAL"] == "AC")]
# Two possibilities 1959 and 1963, find the most common year and use that 
search59 = pipes[(pipes["DIAMETER"] == 6) & (pipes["MATERIAL"] == "AC") & (pipes["INSTYEAR"] == 1959)]
search63 = pipes[(pipes["DIAMETER"] == 6) & (pipes["MATERIAL"] == "AC") & (pipes["INSTYEAR"] == 1963)]
print(break_1_24_08)
print(near1)
print(search59)
print(search63)
# End with using 1963 as install year

       FID_WA_Mai  TYPE  LENGTH  DIAMETER MATERIAL  INSTYEAR MUSYM  Dist_Rail  \
22567        3719  DIST     3.0       8.0       DI    2008.0   AgC       8000   
22568        3720  DIST     4.0       8.0       DI    2008.0   AgC       8000   
23946        3992  DIST    15.0       8.0       DI    2008.0   AgC       8000   
23948        3993  DIST     3.0       8.0       DI    2008.0   AgC       8000   
26885        4497  DIST     3.0       8.0       DI    2008.0   AgC       8000   
28129        4706  DIST   102.0       8.0       DI    2008.0   AgC       8000   
30433        5100  DIST   482.0       8.0       DI    2008.0   AgC       8000   

       FULL_STRNA SPEED           FUNC_CLASS     SLOPE  
22567  NE 73RD ST    25  NEIGHBORHOOD ACCESS  2.000000  
22568  NE 73RD ST    25  NEIGHBORHOOD ACCESS  2.000000  
23946  NE 73RD ST    25  NEIGHBORHOOD ACCESS  1.000000  
23948  NE 73RD ST    25  NEIGHBORHOOD ACCESS  1.000000  
26885  NE 73RD ST    25  NEIGHBORHOOD ACCESS  1.642857  
28129  NE